# When you have a pair that is CA1 or DG, how often is each electrode in each region? Or white matter?

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                         MFG_labels, IFG_labels, nonHPC_MTL_labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'FR1' # 'FR1' 'catFR1' 'RepFR1'

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [2]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
elif exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
elif exp == 'RepFR1':
    exp_df = exp_df[
                    (df.subject!='R1564J') # clearly something wrong with these EEG when looking at ripple raster
                    ]
exp_df.shape

exp_df = exp_df[540:] # for FR1 this is R1385E-onwwards

exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(565, 17)

,Recognition,all_events,contacts,experiment,import_type,localization,math_events,montage,original_experiment,original_session,pairs,ps4_events,session,subject,subject_alias,system_version,task_events
2106,NaN,protocols/r1/subjects/R1449T/experiments/FR1/s...,protocols/r1/subjects/R1449T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1449T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1449T/localizations/0/m...,NaN,0,R1449T,R1449T,3.4,protocols/r1/subjects/R1449T/experiments/FR1/s...
2107,NaN,protocols/r1/subjects/R1449T/experiments/FR1/s...,protocols/r1/subjects/R1449T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1449T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1449T/localizations/0/m...,NaN,1,R1449T,R1449T,3.4,protocols/r1/subjects/R1449T/experiments/FR1/s...
2123,NaN,protocols/r1/subjects/R1454M/experiments/FR1/s...,protocols/r1/subjects/R1454M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1454M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1454M/localizations/0/m...,NaN,0,R1454M,R1454M,3.4,protocols/r1/subjects/R1454M/experiments/FR1/s...
2136,NaN,protocols/r1/subjects/R1457T/experiments/FR1/s...,protocols/r1/subjects/R1457T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1457T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1457T/localizations/0/m...,NaN,0,R1457T,R1457T,3.4,protocols/r1/subjects/R1457T/experiments/FR1/s...
2137,NaN,protocols/r1/subjects/R1457T/experiments/FR1/s...,protocols/r1/subjects/R1457T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1457T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1457T/localizations/0/m...,NaN,1,R1457T,R1457T,3.4,protocols/r1/subjects/R1457T/experiments/FR1/s...
2139,NaN,protocols/r1/subjects/R1459M/experiments/FR1/s...,protocols/r1/subjects/R1459M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1459M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1459M/localizations/0/m...,NaN,0,R1459M,R1459M,3.4,protocols/r1/subjects/R1459M/experiments/FR1/s...
2140,NaN,protocols/r1/subjects/R1459M/experiments/FR1/s...,protocols/r1/subjects/R1459M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1459M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1459M/localizations/0/m...,NaN,6,R1459M,R1459M,3.4,protocols/r1/subjects/R1459M/experiments/FR1/s...
2145,NaN,protocols/r1/subjects/R1460M/experiments/FR1/s...,protocols/r1/subjects/R1460M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1460M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1460M/localizations/0/m...,NaN,0,R1460M,R1460M,3.4,protocols/r1/subjects/R1460M/experiments/FR1/s...
2146,NaN,protocols/r1/subjects/R1461T/experiments/FR1/s...,protocols/r1/subjects/R1461T/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1461T/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1461T/localizations/0/m...,NaN,0,R1461T,R1461T,3.4,protocols/r1/subjects/R1461T/experiments/FR1/s...
2154,NaN,protocols/r1/subjects/R1462M/experiments/FR1/s...,protocols/r1/subjects/R1462M/localizations/0/m...,FR1,build,0,protocols/r1/subjects/R1462M/experiments/FR1/s...,0,NaN,NaN,protocols/r1/subjects/R1462M/localizations/0/m...,NaN,0,R1462M,R1462M,3.4,protocols/r1/subjects/R1462M/experiments/FR1/s...


In [3]:
# load df from hamming run so can take only trials that loaded from it

# # save df code for each detector type
# staresina_ca1_df = pd.DataFrame(data={'session':session_name_array,'subject':subject_name_array,'channel_num':channel_nums_array})
# staresina_ca1_df.to_csv('/home1/john/SWR/compiled/staresina_ca1_df.csv') 

hamming_df = pd.read_csv('/home1/john/SWR/compiled/hamming_df.csv') 
hamming_df = hamming_df.drop_duplicates().reset_index(drop=True)
# change session value to int of the session number
for tr in range(len(hamming_df)):
    hamming_df.at[tr,'session'] = int(hamming_df.iloc[tr].session[7:])
hamming_df

,subject,session,channel_num
0,R1001P,0,21
1,R1001P,0,22
2,R1001P,0,57
3,R1001P,1,21
4,R1001P,1,22
...,...,...,...
1466,R1467M,0,16
1467,R1467M,0,17
1468,R1467M,0,18
1469,R1467M,0,19


In [60]:
def getBestRegionNames(contact_pairs,contact_localization):

    # go through this list order and select the region from the best atlas available

    if 'stein.region' in contact_pairs: # if 'stein' in contact_row.dtype.names:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan'):
            region = test_region.lower()
            atlas_type = 'stein'

    if 'das.region' in contact_pairs:
        test_region = str(contact_pairs['stein.region'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan'):
            region = test_region.lower()
            atlas_type = 'das'

    if 'atlases.mtl' in contact_localization:    
        test_region = str(contact_localization['atlases.mtl'].values[0])
        if (test_region is not None) and (len(test_region)>1) and \
           (test_region not in 'None') and (test_region != 'nan'):
            regs.append(test_region.lower())
            atlas_type.append('MTL_localization')

#         else:
#             pass
#     if len(localizations) > 1:             # 'whole_brain' from localization.json
#         if loc_wb_names[pair_ct] != '' and loc_wb_names[pair_ct] != ' ':
#             if str(loc_wb_names[pair_ct]) != 'nan': # looking for "MTL" field in localizations.json
#                 regs.append(loc_wb_names[pair_ct].lower())
#                 atlas_type.append('wb_localization')
#                 continue
#             else:
#                 pass
#         else:
#             pass
#     if 'wb.region' in contact_row:
#         test_region = str(contact_row['wb.region'])
#         if (test_region is not None) and (len(test_region)>1) and \
#            (test_region not in 'None') and (test_region != 'nan'):
#             regs.append(test_region.lower())
#             atlas_type.append('wb')
#             continue
#         else:
#             pass
#     if len(localizations) > 1:             # 'dk' from localization.json
#         if loc_dk_names[pair_ct] != '' and loc_dk_names[pair_ct] != ' ':
#             if str(loc_dk_names[pair_ct]) != 'nan': # looking for "dk" field in localizations.json
#                 regs.append(loc_dk_names[pair_ct].lower())
#                 atlas_type.append('dk_localization')
#                 continue
#             else:
#                 pass
#         else:
#             pass
#     if 'dk.region' in contact_row:
#         test_region = str(contact_row['dk.region'])
#         if (test_region is not None) and (len(test_region)>1) and \
#            (test_region not in 'None') and (test_region != 'nan'):
#             regs.append(test_region.lower())
#             atlas_type.append('dk')
#             continue
#         else:
#             pass
#     if 'ind.corrected.region' in contact_row: # I don't think this ever has a region label but just in case
#         test_region = str(contact_row['ind.corrected.region'])
#         if (test_region is not None) and (len(test_region)>1) and \
#            (test_region not in 'None') and (test_region not in 'nan'):
#             regs.append(test_region.lower())
#             atlas_type.append('ind.corrected')
#             continue
#         else:
#             pass  
#     if 'ind.region' in contact_row:
#         test_region = str(contact_row['ind.region'])
#         if (test_region is not None) and (len(test_region)>1) and \
#            (test_region not in 'None') and (test_region != 'nan'):
#             regs.append(test_region.lower())
#             atlas_type.append('ind')
#             # [tal_struct[i].atlases.ind.region for i in range(len(tal_struct))] # if you want to see ind atlases for comparison to above
#             # have to run this first though to work in pdb dubugger: globals().update(locals())                  
#             continue
#         else:
#             regs.append('No atlas')
#             atlas_type.append('No atlas')
#     else: 
#         regs.append('No atlas')
#         atlas_type.append('No atlas')
                
    return regs, atlas_type, pair_number

In [61]:
for row in exp_df.itertuples(): #sub_df.itertuples():   
    
    if sum( (row.subject==hamming_df.subject) & (row.session==hamming_df.session) )>0: # if this session is kept
        
        # get the HPC channels to target
        analyzed_channels = hamming_df[(row.subject==hamming_df.subject) & (row.session==hamming_df.session)].channel_num.values
    
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
            
        pairs = reader.load('pairs')
        try:
            localizations = reader.load('localization')
#             localization_exists = 1
        except:
            localizations = []
        
        tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
        elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs)

        # for each pair, get the two contacts, and then figure out what their region is. Then save both in a list        
        contacts = reader.load('contacts')
        
        for channel in analyzed_channels:
            pair_label = pairs.iloc[channel].label
            split_pair = pair_label.split('-')
            
            for contact in split_pair:
                contact_pairs = contacts.iloc[findInd(contacts.label==contact)]
                contact_localization = localizations.iloc[findInd(localizations.iloc[:,0]['contacts'].index.values==contact)]

                region,_,_ = getBestRegionNames(contact_pairs,contact_localization)
                


In [62]:
region

['"dg"']

In [ ]:
contacts['stein.regoin']